In [1]:
from gudiff_model import PDBDataSet_GraphCon
from gudiff_model.Graph_UNet import GraphUNet
from data_rigid_diffuser.diffuser import FrameDiffNoise

from se3_transformer.model.fiber import Fiber
import torch
import os
import logging
from datetime import datetime
from collections import defaultdict
import time
import tree
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from torch import einsum
import numpy as np
import se3_diffuse.utils as du
import util.framediff_utils as fu
from data_rigid_diffuser import rigid_utils as ru
import copy
import util.pdb_writer 
from experiment.Experiment import Experiment
import json
import argparse




In [2]:
#mode meta

import pandas as pd

In [5]:
ld = torch.load('/mnt/c/Users/nwood/OneDrive/Desktop/GraphUNet_SE3Diff/GUN_checkpoints/pdb200/08D_09M_2024Y_00h_20m_15s/step_17760.pth')

In [6]:
ld['conf']

{'batch_size': 4,
 'topk': 4,
 'stride': 4,
 'KNN': 30,
 'num_heads': 8,
 'channels': 32,
 'channels_div': 4,
 'nodefeats_0': 32,
 'nodefeats_1': 6,
 'num_layers': 1,
 'num_layers_ca': 2,
 'edge_feature_dim': 1,
 'latent_pool_type': 'avg',
 't_size': 12,
 'min_t': 0.001,
 'max_t': 1.0,
 'mult': 2,
 'zero_lin': True,
 'use_tdeg1': True,
 'cuda': True,
 'learning rate': 0.0005,
 'weight_decay': 5e-06,
 'device': 'cuda',
 'num_epoch': 120,
 'log_freq': 1000,
 'ckpt_freq': 5000,
 'early_chkpt': 2,
 'coord_scale': 10.0,
 'dataset_max': 5000,
 'meta_data_path': '/mnt/h/datasets/p200/metadata.csv',
 'sample_mode': 'length_batch',
 'ckpt_dir': 'GUN_checkpoints/',
 'eval_dir': 'Eval_Direc/',
 'trained_steps': 17760,
 'epoch': 17760}

In [3]:
fn = '/mnt/h/datasets/p200/metadata.csv'
dir_path = os.path.dirname(os.path.realpath(fn))
df = pd.read_csv(fn)

dirpath = '/gdrive_Test/'

In [4]:
os.path.abspath(os.path.join(os.path.dirname(fn)))

'/mnt/h/datasets/p200'

In [8]:
_dataset_indices = list(range(len(df)))
df['index'] = _dataset_indices

In [ ]:
chain_feats = self._process_csv_row(processed_file_path,index=idx)

In [9]:
sampled_order = df.groupby('modeled_seq_len').sample(8, replace=True, random_state=0) #one batch per length


In [17]:
sampled_order

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration,index
5826,4ttk,/mnt/h/datasets/p200/tt/4ttk.pkl,/mnt/h/datasets/mmCIF/tt/4ttk.cif,1,monomeric,1.25,x-ray diffraction,1,homomer,21,1,12.000,0.000,2.000,0.715255,5826
3967,6mw0,/mnt/h/datasets/p200/mw/6mw0.pkl,/mnt/h/datasets/mmCIF/mw/6mw0.cif,1,monomeric,0.78,x-ray diffraction,1,homomer,12,1,4.000,0.000,0.000,0.487701,3967
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
4701,6phq,/mnt/h/datasets/p200/ph/6phq.pkl,/mnt/h/datasets/mmCIF/ph/6phq.cif,1,monomeric,1.32,x-ray diffraction,1,homomer,46,1,3.000,23.000,0.000,1.174012,4701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,8buw,/mnt/h/datasets/p200/bu/8buw.pkl,/mnt/h/datasets/mmCIF/bu/8buw.cif,"2,2","dimeric,dimeric",2.80,x-ray diffraction,4,heteromer,200,200,0.480,0.135,0.385,1.865659,649
649,8buw,/mnt/h/datasets/p200/bu/8buw.pkl,/mnt/h/datasets/mmCIF/bu/8buw.cif,"2,2","dimeric,dimeric",2.80,x-ray diffraction,4,heteromer,200,200,0.480,0.135,0.385,1.865659,649
2701,1hvi,/mnt/h/datasets/p200/hv/1hvi.pkl,/mnt/h/datasets/mmCIF/hv/1hvi.cif,2,dimeric,1.80,x-ray diffraction,2,heteromer,200,200,0.395,0.040,0.555,1.729709,2701
2701,1hvi,/mnt/h/datasets/p200/hv/1hvi.pkl,/mnt/h/datasets/mmCIF/hv/1hvi.cif,2,dimeric,1.80,x-ray diffraction,2,heteromer,200,200,0.395,0.040,0.555,1.729709,2701


In [13]:
df.iloc[5826]

pdb_name                                            4ttk
processed_path          /mnt/h/datasets/p200/tt/4ttk.pkl
raw_path               /mnt/h/datasets/mmCIF/tt/4ttk.cif
oligomeric_count                                       1
oligomeric_detail                              monomeric
resolution                                          1.25
structure_method                       x-ray diffraction
num_chains                                             1
quaternary_category                              homomer
seq_len                                               21
modeled_seq_len                                        1
coil_percent                                        12.0
helix_percent                                        0.0
strand_percent                                       2.0
radius_gyration                                 0.715255
index                                               5826
Name: 5826, dtype: object

In [16]:
df.iloc[4701]

pdb_name                                            6phq
processed_path          /mnt/h/datasets/p200/ph/6phq.pkl
raw_path               /mnt/h/datasets/mmCIF/ph/6phq.cif
oligomeric_count                                       1
oligomeric_detail                              monomeric
resolution                                          1.32
structure_method                       x-ray diffraction
num_chains                                             1
quaternary_category                              homomer
seq_len                                               46
modeled_seq_len                                        1
coil_percent                                         3.0
helix_percent                                       23.0
strand_percent                                       0.0
radius_gyration                                 1.174012
index                                               4701
Name: 4701, dtype: object

In [10]:
sampled_order['index']

5826    5826
3967    3967
4701    4701
4701    4701
4701    4701
        ... 
649      649
649      649
2701    2701
2701    2701
5086    5086
Name: index, Length: 1592, dtype: int64

In [11]:
for x in iter(sampled_order['index'].tolist()):
    print(x)

5826
3967
4701
4701
4701
3969
4701
4697
3968
1248
1248
3968
1250
1249
1248
1249
5981
4429
5981
1733
4429
5980
5968
1733
5703
6334
6343
7010
7010
246
5124
5703
5151
3388
2106
2106
1974
4221
3723
6178
976
5400
5353
977
3279
6258
978
4437
1325
679
679
1731
4005
4005
4006
5962
4009
3805
2225
5778
6189
5629
638
638
6177
6266
6266
5667
5989
6177
5247
329
3408
4461
6459
6915
3825
6917
3825
5287
1725
1725
1724
1725
1725
4423
1725
1724
801
4382
4382
4111
4111
4111
243
711
3000
4783
1519
244
3000
244
724
749
2201
912
4534
1592
912
4022
4354
4534
3280
3187
4112
6456
196
4356
1732
4449
6576
5953
7234
1308
6526
1308
1318
7234
4485
3277
4485
4675
1367
3277
4485
1367
4130
2307
5399
4130
6734
4990
2096
4990
4263
4557
691
691
6482
691
6482
4557
2794
4924
100
6330
100
7101
100
7101
2325
2326
2325
6454
899
5406
5406
2326
2278
3855
3855
2278
5805
3901
5805
2278
4299
3412
5358
6188
6186
295
6186
4700
5039
164
5831
5039
4694
4620
6028
3171
2281
247
247
247
4699
4699
2281
4439
7102
7102
771
771
4070
4434
771

In [28]:
fn1 = df['processed_path'].iloc[0]

In [29]:
fn1

'/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [31]:
os.path.basename(os.path.dirname(fn1))

'00'

In [35]:
df['processed_path'] = df['processed_path'].apply(lambda x: os.path.join(  dirpath,  os.path.basename((os.path.dirname(x))),  os.path.basename(x)) )

In [11]:
os.path.basename(x)

In [36]:
df['processed_path'].iloc[0]

'/gdrive_Test/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [15]:
df

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration
0,0001581c72611c299a7297cf4c1d9851_0001,/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.138462,0.861538,0.0,1.111881
1,0002182ba9e87d7b0d6b7d16b7001923_0001,/mnt/h/datasets/bCov_4H/00/0002182ba9e87d7b0d6...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.133914
2,00052a785cf4cffaa60b15d2ccc78f9d_0001_1,/mnt/h/datasets/bCov_4H/00/00052a785cf4cffaa60...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.111930
3,000701cc0c20e04627e1e44f84d5a6b6_0001_1,/mnt/h/datasets/bCov_4H/00/000701cc0c20e04627e...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111956
4,000877380acb95e21726d9df052ec962_0001,/mnt/h/datasets/bCov_4H/00/000877380acb95e2172...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.104479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27889,fff532a1f3bd52993b00d58dc306e14c_0001_1,/mnt/h/datasets/bCov_4H/ff/fff532a1f3bd52993b0...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.090073
27890,fff8daf3a52878a3bf78d4aa4fd68612_0001_1,/mnt/h/datasets/bCov_4H/ff/fff8daf3a52878a3bf7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.169231,0.830769,0.0,1.098638
27891,fffa17b132597274c1fd4fd217829317_0001_1,/mnt/h/datasets/bCov_4H/ff/fffa17b132597274c1f...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111163
27892,fffaaf5372bf5d4f67462faa60fe4466_0001,/mnt/h/datasets/bCov_4H/ff/fffaaf5372bf5d4f674...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.099117


In [13]:
df['processed_path']

0        /gdrive_Test/0001581c72611c299a7297cf4c1d9851_...
1        /gdrive_Test/0002182ba9e87d7b0d6b7d16b7001923_...
2        /gdrive_Test/00052a785cf4cffaa60b15d2ccc78f9d_...
3        /gdrive_Test/000701cc0c20e04627e1e44f84d5a6b6_...
4        /gdrive_Test/000877380acb95e21726d9df052ec962_...
                               ...                        
27889    /gdrive_Test/fff532a1f3bd52993b00d58dc306e14c_...
27890    /gdrive_Test/fff8daf3a52878a3bf78d4aa4fd68612_...
27891    /gdrive_Test/fffa17b132597274c1fd4fd217829317_...
27892    /gdrive_Test/fffaaf5372bf5d4f67462faa60fe4466_...
27893    /gdrive_Test/fffab47f11ab9cbb43a0143b90b3e454_...
Name: processed_path, Length: 27894, dtype: object

In [ ]:
with open('configs/base_gun.json','r') as f:
    conf = json.load(f)



if __name__ == '__main__':
    parser = argparse.ArgumentParser('Train Graph U-Net')
    parser.add_argument('name', help='name of run',type=str)

    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("-c", '--config_path',  help='path to config file in json format',type=str)
    group.add_argument("-k", '--checkpoint_file', help='previous checkpoint to load',type=str)

    args = parser.parse_args()

    if args.checkpoint_file is not None:

        ckpt_model=torch.load(args.checkpoint_file)['model']
        ckpt_opt = torch.load(args.checkpoint_file)['optimizer']
        conf = torch.load(args.checkpoint_file)['conf']

        exp = Experiment(conf,name=args.name, ckpt_model=ckpt_model, ckpt_opt=ckpt_opt)

    else:
        with open(args.config_path, 'r') as f:
            conf = json.load(f)
        exp = Experiment(conf, name=args.name, ckpt_model=None, ckpt_opt=None)

    tl, vl = exp.create_dataset() #load dataset from 'meta_data_path.csv'
    exp.start_training()